In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.4 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=3d3a70ffe6d53678bfce5118ae3fde5c6ac325d3aff8e662abb97488a1272622
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.9 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 a0601.csv		       heatmap529_UK.ipynb
 csv			       heatmap530.ipynb
 csv.zip		       Heat_Map_Online_language.ipynb
 dataframe_csv		       __MACOSX
 dataframe_csv_EU	       mapping_UK
 dataframe_csv_UK	       mapping_US
 dataframe_result_529_UK.csv   non_violant1.csv
 dataframe_result_530_UK.csv   non_violant2.csv
 dataframe_result_531_UK.csv   UK
 dataframe_result_csv	       violant.csv
 dataframe_result_csv_EU      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_601 = 'csv/US05_29.csv'

In [8]:
ct = pd.read_csv(ct_path_601)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071
1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941
2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568
3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173
4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22:18:59,en,0,1486517|10157744314187144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23895,203828796317254_3382699755096793,Faceb

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}"
1,{'America': 1},{}
2,{},{}
3,{},{}
4,{},{}


In [24]:
ct.to_csv('a0529.csv')

In [25]:
ct=pd.read_csv('a0529.csv')

In [ ]:
!ls

In [ ]:
###
ct=pd.read_csv('a0529.csv')

In [26]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,104380826317363_2958316794257071,Facebook,2020-05-29 22:10:01,2021-06-15 16:21:42,link,Aftershocks of George Floyd's death hit Centra...,theledger.com,George Floyd attended South Florida State Coll...,George Floyd attended South Florida State Coll...,1.0,https://www.theledger.com/news/20200529/afters...,https://www.facebook.com/104380826317363/posts...,3375,-1.250000,1.0,5,7,0,0,2,0,2,0,0,0,3,2,2,2,2,3,2,4,0,1,23719,News Chief,newschief,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The News Chief is a daily newspaper in Winter ...,False,2011-04-27 15:29:39,en,0,23719|2958316794257071,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}",{}
1,1,2015249512117888_2353472281628941,Facebook,2020-05-29 22:10:00,2021-07-14 01:56:00,link,Justice — Zina for Congress,zinaforcongress.com,Protesters taking to the streets to seek justi...,"Ahmaud Arbery, Breonna Taylor and George Floyd...",2.0,http://www.zinaforcongress.com/justice,https://www.facebook.com/2015249512117888/post...,796,1.826087,1.0,31,6,0,5,0,0,0,0,0,1,6,2,2,3,2,3,2,3,0,2,8581055,Zina Spezakis for Congress NJ9,zspezakisNJ,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_page,US,Our leadership is ineffective and corrupt. It ...,False,2019-02-12 20:21:37,en,0,8581055|2353472281628941,{'America': 1},{},{}
2,2,127784800585461_3342576742439568,Facebook,2020-05-29 22:10:00,2021-07-20 14:56:16,link,A Fearless Response To George Floyd’s Death | ...,fearlessfollower.org,A Fearless Response To George Floyd’s Death by...,Hey Fearless Followers! Here's a challenging r...,1.0,https://fearlessfollower.org/a-fearless-respon...,https://www.facebook.com/127784800585461/posts...,4484,3.263158,1.0,16,29,4,11,0,0,2,0,0,0,3,3,2,3,0,0,0,0,0,1,1467090,Ridge Point Community Church,ridgepointchurch,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Ridge Point exists to Follow Jesus and Fearles...,False,2010-06-14 17:44:07,en,0,1467090|3342576742439568,{},{},{}
3,3,7127779172_10158347173219173,Facebook,2020-05-29 22:09:59,2021-07-19 06:12:54,link,The Shooting Of Black Americans Started Long B...,pridenation.com,By Taryn Finley | HuffPost Donald Trump threat...,The Shooting Of Black Americans Started Long B...,2.0,http://ow.ly/8IiY50zUbyT,https://www.facebook.com/7127779172/posts/1015...,2459,-0.900000,1.0,0,0,0,0,0,0,1,0,0,0,1,2,1,1,1,1,2,1,0,1,8609875,PNN Media Group,pridenation,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,PHARMATIVE PNN Media Group is the parent compa...,False,2008-01-07 06:37:40,en,0,8609875|10158347173219173,{},{},{}
4,4,57218502143_10157744314187144,Facebook,2020-05-29 22:09:54,2021-08-06 00:24:46,link,CN2 Newscast 5/29/20 - CN2 News,cn2.com,A pastor spreading a message of unity and love...,A pastor spreading a message of unity and love...,2.0,https://www.cn2.com/cn2-newscast-5-29-20/,https://www.facebook.com/57218502143/posts/101...,24960,-6.000000,1.0,7,0,0,0,0,0,0,0,0,0,8,6,3,4,4,4,4,9,0,3,1486517,CN2 News,CN2RockHill,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,NEWS TIP? 803-326-2762 or news@cn2.com\n\n,False,2009-03-09 22:18:59,en,0,1486517|10157744314187144,{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,

In [27]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [28]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [29]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Avon Park': 1, 'Polk County': 1, 'Minneapoli...","{'Avon Park': 1, 'Polk County': 1}",{}
1,{'America': 1},{},{}
6,"{'Minneapolis': 1, 'Roseville': 1}",{},{}
8,{'Houston': 1},{'Minneapolis': 1},{'Houston': 1}
9,{},{'Minneapolis': 1},{'Houston': 1}


In [30]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [31]:
ct_ner['NER-msg']

0        [Avon Park, Polk County, Minneapolis]
1                                    [America]
6                     [Minneapolis, Roseville]
8                                    [Houston]
9                                           []
                         ...                  
23894                                       []
23895                            [Minneapolis]
23896                                     [US]
23897           [Minneapolis, Hennepin County]
23899                                       []
Name: NER-msg, Length: 19484, dtype: object

In [32]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
ct_ner['NER']

0        [Avon Park, Polk County, Minneapolis, Avon Par...
1                                                [America]
6                                 [Minneapolis, Roseville]
8                          [Houston, Minneapolis, Houston]
9                                   [Minneapolis, Houston]
                               ...                        
23894                           [Minneapolis, Minneapolis]
23895              [Minneapolis, Minneapolis, Minneapolis]
23896                       [US, Minneapolis, Minneapolis]
23897    [Minneapolis, Hennepin County, Minneapolis, Mi...
23899                                        [Minneapolis]
Name: NER, Length: 19484, dtype: object

In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
ner_lst = ct_ner['NER'].values

In [38]:
ner_lst

array([list(['avon park', 'polk county', 'minneapolis']),
       list(['america']), list(['minneapolis', 'roseville']), ...,
       list(['minneapolis', 'us']),
       list(['minneapolis', 'hennepin county']), list(['minneapolis'])],
      dtype=object)

In [39]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [40]:
list

['avon park',
 'polk county',
 'minneapolis',
 'america',
 'minneapolis',
 'roseville',
 'minneapolis',
 'houston',
 'minneapolis',
 'houston',
 'minneapolis',
 'minneapolis',
 'minnesota',
 'minneapolis',
 'minneapolis',
 'the united states',
 'fontana',
 'minnesota',
 'minneapolis',
 'minneapolis',
 'oklahoma county',
 'minnesota',
 'oklahoma',
 'okla.',
 'oklahoma county',
 'minneapolis',
 'minneapolis',
 'hennepin county',
 'minneapolis',
 'hennepin county',
 'minneapolis',
 'minnesota',
 'the united states',
 'minneapolis',
 'minneapolis',
 'minnesota',
 'atlanta',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minnesota',
 'ohio',
 'the united states',
 'minnesota',
 'ohio',
 'minneapolis',
 'minneapolis',
 'portland',
 'minneapolis',
 'us',
 'scranton',
 'u.s.',
 'houma',
 'minneapolis',
 'the united states',
 'kansas city',
 'kansas',
 'thibodaux',
 'oklahoma city',
 'st. paul',
 'washington',
 '

In [41]:
c = collections.Counter()

In [42]:
dict_a=collections.Counter(list).most_common()

In [43]:
dict_a

[('minneapolis', 16071),
 ('minnesota', 4009),
 ('america', 1044),
 ('st. paul', 919),
 ('hennepin county', 761),
 ('the united states', 531),
 ('denver', 515),
 ('u.s.', 492),
 ('us', 463),
 ('houston', 379),
 ('columbus', 373),
 ('louisville', 276),
 ('mn', 247),
 ('atlanta', 217),
 ('mississippi', 208),
 ('chicago', 206),
 ('south minneapolis', 200),
 ('illinois', 192),
 ('new york', 185),
 ('georgia', 184),
 ('phoenix', 176),
 ('u.s', 172),
 ('ohio', 171),
 ('washington', 159),
 ('los angeles', 158),
 ('kentucky', 145),
 ('colorado', 136),
 ('nyc', 113),
 ('lake county', 112),
 ('new york city', 105),
 ('minn.', 105),
 ('milwaukee', 104),
 ('nj', 103),
 ('florida', 101),
 ('third ward', 100),
 ('miss.', 94),
 ('joliet', 91),
 ('american city', 83),
 ('boston', 76),
 ('baltimore', 74),
 ('tennessee', 74),
 ('texas', 73),
 ('memphis', 71),
 ('california', 70),
 ('san jose', 67),
 ('orlando', 66),
 ('miami', 62),
 ('birmingham', 59),
 ('manhattan', 58),
 ('seattle', 58),
 ('portland',

In [44]:
dataframe601=pd.DataFrame(dict_a)

In [45]:
len(dict_a)

1559

In [46]:
dataframe601.to_csv('dataframe0529.csv')